In [1]:
import glob
import re
import json
import itertools
import pandas as pd
from pathlib import Path
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, BatchNormalization, Activation, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle

from tqdm import tqdm

from sklearn.model_selection import GridSearchCV
import joblib
from sklearn.model_selection import TimeSeriesSplit

# all the functions from helpers.py
from helpers_scenario2_redes import *

In [2]:
annotations_folder = '../data/raw/scenario_2/fold_0/train/annotations/'
# physiology_folder = "../data/preprocessed/cleaned/scenario_1/fold_0/train/physiology/" #'../data/raw/scenario_1/train/physiology/'
physiology_folder = "../data/preprocessed/cleaned_and_prepro_improved/scenario_2/fold_0/train/physiology/" #'../data/raw/scenario_1/train/physiology/'data\preprocessed\

df_physiology = load_read_and_append_csvs(physiology_folder)
df_annotations = load_read_and_append_csvs(annotations_folder)

videos = df_physiology.video.unique()
subjects = df_physiology.subject.unique()

splits = split_subjects_train_test(subjects, 3)

In [3]:
videos = [0]

In [6]:
from keras.layers import Dropout
from keras.regularizers import l1

# Create the CNN model
def create_cnn_model(input_shape):
    input_signal = Input(shape=input_shape)

    x = Conv1D(16, 5, padding='same')(input_signal)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)

    x = Conv1D(8, 3, padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)

    x = Flatten()(x)
    f_peripheral = Dense(64, activation='relu', kernel_regularizer=l1(0.001))(x)

    return Model(inputs=input_signal, outputs=f_peripheral)

In [5]:
from concurrent.futures import ProcessPoolExecutor, as_completed, ThreadPoolExecutor
import threading
import multiprocessing
from tqdm import tqdm
import itertools

# Wrap the outer loop with tqdm
for video in tqdm(videos, desc="Processing videos", unit="video", bar_format="{l_bar}%s{bar}%s{r_bar}" % ('\033[32m', '\033[0m')):
    print(f"Processing video: {video}")

    df_physiology_video = df_physiology.loc[df_physiology.video == video]
    df_annotations_video = df_annotations.loc[df_annotations.video == video]

    del df_annotations
    del df_physiology

    rmses = []

    for split in splits:
        # print(split)

        X_train, X_test, y_train, y_test, numeric_column_indices, categorical_column_indices = preprocess(
            df_physiology_video.copy(), df_annotations_video.copy(), split=split, predictions_cols=['arousal', 'valence'], aggregate=None,
            window_duration=10000, resample_rate=100)

        del df_annotations_video
        del df_physiology_video

        # Extract arousal and valence values from y_train and y_test
        y_arousal_train = y_train[:, 0]
        y_valence_train = y_train[:, 1]
        y_arousal_test = y_test[:, 0]
        y_valence_test = y_test[:, 1]

        input_shape = (X_train.shape[1], X_train.shape[2])  # Updated input shape
        cnn_model = create_cnn_model(input_shape)

        # Create separate heads for valence and arousal prediction
        valence_output = Dense(1, activation='linear', name='valence_output')(cnn_model.output)
        arousal_output = Dense(1, activation='linear', name='arousal_output')(cnn_model.output)

        # Combine the model
        final_model = Model(inputs=cnn_model.input, outputs=[valence_output, arousal_output])

        # Compile the model
        final_model.compile(optimizer='adam',
                            loss={'valence_output': 'mse',
                            'arousal_output': 'mse'},
                            metrics=[tf.keras.metrics.RootMeanSquaredError()])

        # Set up early stopping
        early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

        # Train the model
        history = final_model.fit(X_train, {'valence_output': y_valence_train, 'arousal_output': y_arousal_train},
                                validation_split=0.2, epochs=2, batch_size=32,
                                callbacks=[early_stopping_callback])
        
        print(final_model.evaluate(X_test, [y_valence_test, y_arousal_test]))

        #rmse = ...

        #rmses.append(rmse)

    #average_rmse = np.mean(rmses, axis=0)

    # CREAR UNA FORMA DE GUARDAR CORRIDAS EN JSON


Processing videos:   0%|          | 0/1 [00:00<?, ?video/s]

Processing video: 0
Epoch 1/50


Processing videos:   0%|          | 0/1 [03:18<?, ?video/s]


ResourceExhaustedError: Graph execution error:

OOM when allocating tensor with shape[12,41085,1000] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator cpu
	 [[{{node GatherV2}}]]
	 [[IteratorGetNext]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_3268]

In [19]:
print(final_model.evaluate(X_test, {'valence_output': y_valence_test, 'arousal_output': y_arousal_test}))


ValueError: Data cardinality is ambiguous:
  x sizes: 1000
  y sizes: 20543, 20543
Make sure all arrays contain the same number of samples.